# Uso de Python para construir el modelo

La idea es usar python para modificar el archivo de excel.

In [10]:
import pandas as pd
import numpy as np
# Cargar las hojas del excel como data frame separados
file_xls= '../Data/modelData.xls'
PstKB_OF_sheet='PstKB_OF'
PstKB_OF_columns = ['NaN','ID','KB','Reaction Name','Molecule','NaN']
PstKB_OF = pd.read_excel(file_xls,sheet_name=PstKB_OF_sheet, header=None, names=PstKB_OF_columns)
biomass3_sheet ='biomass3'
biomass3_columns = ['KB','Changes Guevara']
biomass3 = pd.read_excel(file_xls,sheet_name=biomass3_sheet,names=biomass3_columns) 

# Dividir las columnas 'KB' y 'Changes Guevara' y crear nuevas columnas
split_columns = biomass3['KB'].str.split(expand=True)
biomass3['Metabolito'] = split_columns[0]  # Assuming the Metabolito column is the first one after the split
biomass3['KB'] = split_columns[1]
split_columns = biomass3['Changes Guevara'].str.split(expand=True)
biomass3['Changes Metabolito'] = split_columns[0]  # Assuming the Changes Metabolito column is the first one after the split
biomass3['Changes Guevara'] = split_columns[1]
biomass3.columns = ['Metabolito','Metabolito-G','KB','Guevara']
# Fusionar las columnas 'Metabolito' y 'Metabolito-G' en una nueva columna llamada 'Metabolito-Total'
biomass3['ID'] = biomass3['Metabolito'].combine_first(biomass3['Metabolito-G'])
# Eliminar las columnas originales 'Metabolito' y 'Metabolito-G' si así lo deseas
biomass3 = biomass3.drop(['Metabolito', 'Metabolito-G'], axis=1)
# Reorganizar las columnas del DataFrame
biomass3 = biomass3[['ID', 'KB', 'Guevara']]

In [3]:
extractSec = PstKB_OF[['ID','Reaction Name']]
mergeDf = pd.merge(extractSec, biomass3, on='ID', how='inner')
mergeDf.columns

Index(['ID', 'Reaction Name', 'KB', 'Guevara'], dtype='object')

## Cargar la tabla comparativa
Dentro del archivo hay una tabla comparativa entre 3 modelos Botero, Guevara y PstKB

In [4]:
Comparative_sheet='Comparative OFs'
Comparative_colums = ['Metabolito', 'Guevara 2023','Botero 2018','PstKB_Ori','PstKB_cobra']
comp_Table= pd.read_excel(file_xls,sheet_name=Comparative_sheet,names=Comparative_colums)
comp_Table = comp_Table[['Metabolito','Botero 2018','Guevara 2023','PstKB_Ori','PstKB_cobra']]
comp_Table

,Metabolito,Botero 2018,Guevara 2023,PstKB_Ori,PstKB_cobra
cpd00001[c0],H2O,52.486878,NaN,30.000000,52.486878
cpd00002[c0],ATP,50.864030,1.000,30.027000,50.864030
cpd00003[c0],NAD,NaN,NaN,0.000300,0.000300
cpd00004[c0],NADH,NaN,NaN,0.000150,0.000150
cpd00005[c0],NADPH,NaN,NaN,0.000100,0.000100
...,...,...,...,...,...
cpd16443[c0],beta-D-Ribofuranose,NaN,NaN,0.037700,0.037700
cpd16503[c0],Plastoquinol-9,NaN,NaN,0.000155,0.000155
cpd19001[c0],alpha-D-Glucose,0.053000,0.053,1.590000,0.053000
cpd25914[c0],ubiquinol(9),NaN,NaN,0.000100,0.000100


# Crear la Tabla "Rosseta"

crear el indice entre Metablolito - Reaccion Name, para poder unificar las tablas, y luego poder remplazar los valores

In [7]:
# Configurar pandas para mostrar todas las filas
pd.set_option('display.max_rows', None)
# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)
# Restaurar la configuración predeterminada al final de la celda
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [5]:
# Limpiar la columna "Reaction Name"
mergeDf['Reaction Name'] = mergeDf['Reaction Name'].str.replace(' ', '').str.replace(',', '').str.replace('_c0', '')

mergeDf

,ID,Reaction Name,KB,Guevara
0,cpd00001[c0],H2O,30,52.486
1,cpd00003[c0],NAD,0.0003,0.0003
2,cpd00002[c0],ATP,30.027,50.864
3,cpd00006[c0],NADP,0.00013,0.00013
4,cpd00004[c0],NADH,0.00015,0.00015
...,...,...,...,...
68,cpd00012[c0],PPi,0.218,0.218
69,cpd00067[c0],H+,30,30
70,cpd00008[c0],ADP,30,30
71,cpd00014[c0],UDP,0.766,0.766


In [9]:
# Install 
#%pip install fuzzywuzzy
from fuzzywuzzy import fuzz
# Crear una función para encontrar la mejor coincidencia con fuzzywuzzy
def find_best_match(string_to_match, candidates):
    if isinstance(string_to_match, float):
        return None, 0  # Si es un valor float, devuelve valores predeterminados

    best_match = None
    highest_similarity = -1

    for candidate in candidates:
        similarity = fuzz.ratio(string_to_match, candidate)
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = candidate

    return best_match, highest_similarity

# Aplicar la función a cada fila en comp_Table['Metabolito']
comp_Table['Best_Match'], comp_Table['Similarity'] = zip(*comp_Table['Metabolito'].apply(lambda x: find_best_match(x, mergeDf['Reaction Name'])))

# Establecer un umbral de similitud para considerar coincidencias
umbral_similitud = 80

# Filtrar las filas que cumplen con el umbral de similitud
matching_rows = comp_Table[comp_Table['Similarity'] >= umbral_similitud]

# Realizar el merge basado en las coincidencias
comparacion_de_datos = pd.merge(mergeDf, matching_rows, how='left', left_on='Reaction Name', right_on='Best_Match', suffixes=('', '_comp'))

# Agregar las filas que no encontraron coincidencias
filas_sin_coincidencia = comp_Table[comp_Table['Similarity'] < umbral_similitud].copy()
filas_sin_coincidencia['Reaction Name'] = None  # Establecer la columna de coincidencia como None para indicar falta de coincidencia
comparacion_de_datos = pd.concat([comparacion_de_datos, filas_sin_coincidencia], ignore_index=True)

# Eliminar columnas temporales
comparacion_de_datos = comparacion_de_datos.drop(['Best_Match', 'Similarity'], axis=1)

# Imprimir el DataFrame resultante
comparacion_de_datos


Note: you may need to restart the kernel to use updated packages.


,ID,Reaction Name,KB,Guevara,Metabolito,Botero 2018,Guevara 2023,PstKB_Ori,PstKB_cobra
0,cpd00001[c0],H2O,30,52.486,H2O,52.486878,NaN,30.00000,52.486878
1,cpd00003[c0],NAD,0.0003,0.0003,NAD,NaN,NaN,0.00030,0.000300
2,cpd00002[c0],ATP,30.027,50.864,ATP,50.864030,1.0,30.02700,50.864030
3,cpd00006[c0],NADP,0.00013,0.00013,NADP,NaN,NaN,0.00013,0.000130
4,cpd00004[c0],NADH,0.00015,0.00015,NADH,NaN,NaN,0.00015,0.000150
...,...,...,...,...,...,...,...,...,...
68,cpd00012[c0],PPi,0.218,0.218,PPi,0.251164,NaN,0.21800,0.251164
69,cpd00067[c0],H+,30,30,H+,50.821625,NaN,30.00000,50.821625
70,cpd00008[c0],ADP,30,30,ADP,50.821625,30.0,30.00000,50.821625
71,cpd00014[c0],UDP,0.766,0.766,UDP,NaN,NaN,0.76600,0.766000


In [12]:
# Especifica la ruta y nombre del archivo Excel
excel_filename = 'comparacion_de_datos.xlsx'

# Exporta el DataFrame a un archivo Excel
comparacion_de_datos.to_excel(excel_filename, index=False)

In [13]:
# Suponiendo que comparacion_de_datos y PstKB_OF son tus DataFrames

# Realizar el merge utilizando la columna 'ID' como clave
merged_dataframe = pd.merge(comparacion_de_datos, PstKB_OF, on='ID', how='inner')

# Imprimir el DataFrame resultante
merged_dataframe.to_excel('TODO.xlsx',index=False)

In [14]:
# Guardar el nuevo DataFrame en un archivo CSV
merged_dataframe.to_csv('Code.csv', index=False)